In [2]:
from base64 import b64encode, b64decode
import math, random
import numpy as np
from itertools import combinations
from Crypto.Cipher import AES

In [3]:
def xor(a, b):
    return bytes([x^y for (x, y) in zip(a,b)])

### Challenge 9

In [4]:
def pad(msg, block_size):
    extra_bytes = block_size - len(msg) % block_size 
    return msg + bytes([extra_bytes] * extra_bytes)

def unpad(msg):
    return msg[:-msg[-1]]

In [5]:
pad(b"YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

### Challenge 10

In [6]:
def cbc_encrypt(msg, IV, cipher):
    out = bytes([])
    curr_block = IV[:]
    for i in range(len(msg) // 16):
        curr_block = xor(curr_block, msg[i*16:(i+1)*16])
        curr_block = cipher.encrypt(curr_block)
        out += curr_block
    return out

def cbc_decrypt(ciphtext, IV, cipher):
    out = bytes([])
    curr_block = IV[:]
    for i in range(len(ciphtext) // 16):
        dec_block = cipher.decrypt(ciphtext[i*16:(i+1)*16])
        dec_block = xor(curr_block, dec_block)
        out += dec_block
        curr_block = ciphtext[i*16:(i+1)*16]
    return out

In [7]:
key = b"YELLOW SUBMARINE"
IV = bytes([0] * 16)

cipher = AES.new(key, AES.MODE_ECB)

msg = pad(b"Hello, World", 16)

cbc_decrypt(cbc_encrypt(msg, IV, cipher), IV, cipher)

b'Hello, World\x04\x04\x04\x04'

In [8]:
with open('data/10.txt', 'r') as file:
    data = b64decode(''.join(file.readlines()))
    
print(cbc_decrypt(data, IV, cipher).decode('ascii'))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### Challenge 11

In [9]:
def random_bytes(len):
    return bytes([random.randint(0,255) for _ in range(len)])

def encryption_oracle(msg):
    msg = random_bytes(random.randint(5,10)) + msg + random_bytes(random.randint(5,10))
    msg = pad(msg, 16)
    cipher = AES.new(random_bytes(16), AES.MODE_ECB)
    if random.randint(0,1):
        return cipher.encrypt(msg), True
    else:
        return cbc_encrypt(msg, random_bytes(16), cipher), False

In [10]:
def detect_ecb(ciph):
    blocks = len(ciph) // 16
    for i, j in combinations(range(blocks), 2):
        x = xor(ciph[i*16:(i+1)*16], ciph[j*16:(j+1)*16])
        same_bits = 16-np.count_nonzero(bytearray(x))
        if same_bits > 8:
            return True
    return False

In [11]:
for _ in range(100):
    ciph, is_ecb = encryption_oracle(bytes([0]*32*8))
    assert is_ecb == detect_ecb(ciph)

### Challenge 12

**Note**: This attack seems to also work on CBC when the same IV is re-used.

In [12]:
unk_string = ("Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg"
              "aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq"
              "dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg"
              "YnkK")
unk_bytes = b64decode(unk_string)
key = random_bytes(16)
cipher = AES.new(key, AES.MODE_ECB)

def encryption_oracle(msg):
    msg += unk_bytes
    msg = pad(msg, 16)
    return cipher.encrypt(msg)

In [13]:
for i in range(1, 1024):
    len1 = len(encryption_oracle(bytes([0] * i)))
    len2 = len(encryption_oracle(bytes([0] * (i+1))))
    if len1 != len2:
        print(len2 - len1)
        break

16


In [14]:
detect_ecb(encryption_oracle(bytes([0]*32*8)))

True

In [15]:
MAX_UNK_STR_LEN = 1000

msg = b''
for i in range(MAX_UNK_STR_LEN):
    init_pad_count = (15 - i) % 16
    str1 = bytes([0]*init_pad_count)
    for ch in range(256):
        str2 = bytes([0]*init_pad_count) + msg + bytes([ch])
        if encryption_oracle(str1)[init_pad_count+i+1-16:init_pad_count+i+1] == \
           encryption_oracle(str2)[init_pad_count+i+1-16:init_pad_count+i+1]:
                msg += bytes([ch])
                break
print(msg.decode('ascii'))

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  


### Challenge 13

In [26]:
def parse_kv_str(str):
    out = {}
    for pair in str.split('&'):
        key, value = pair.split('=')
        out[key] = value
    return out

def encode_as_kv(dict):
    return '&'.join(['='.join([key, dict[key]]) for key in ['email', 'uid', 'role']])

In [27]:
key = random_bytes(16)
cipher = AES.new(key, AES.MODE_ECB)
num_users = 0

def profile_for(email):
    email = email.replace('&', '').replace('=', '')
    global num_users
    num_users += 1
    res_str = encode_as_kv({'email' : email, 'uid' : '1', 'role' : 'user'}).encode('ascii')
    return cipher.encrypt(pad(res_str, 16))

def decrypt_and_parse(ciph):
    return parse_kv_str(unpad(cipher.decrypt(ciph)).decode('ascii'))

In [28]:
email = 'tom@tomato.com'
c1 = profile_for('tom@tomato.com')[:-16]
c2 = profile_for('0000000000admin' + bytes([11]*11).decode('ascii'))[16:32]
decrypt_and_parse(c1+c2)

{'email': 'tom@tomato.com', 'role': 'admin', 'uid': '1'}

### Challenge 14

**TODO**: review this (can discover length of random string systematically by checking response on blocks of zeroes)

In [19]:
MAX_RAN_STR_LEN = 64

ran_string = random_bytes(random.randint(1,MAX_RAN_STR_LEN))

def encryption_oracle(msg):
    msg = ran_string + msg
    msg += unk_bytes
    msg = pad(msg, 16)
    return cipher.encrypt(msg)

In [20]:
for ran_str_len_guess in range(MAX_RAN_STR_LEN):
    msg = b''
    for i in range(MAX_UNK_STR_LEN):
        init_pad_count = (15 - ran_str_len_guess - i) % 16
        init_total_count = ran_str_len_guess + init_pad_count
        str1 = bytes([0]*init_pad_count)
        for ch in range(256):
            str2 = bytes([0]*init_pad_count) + msg + bytes([ch])
            if encryption_oracle(str1)[init_total_count+i+1-16:init_total_count+i+1] == \
               encryption_oracle(str2)[init_total_count+i+1-16:init_total_count+i+1]:
                    msg += bytes([ch])
                    break
    try:
        if msg[:16] != bytes([0]*16): # hacky
            print(msg.decode('ascii'))
            break
    except UnicodeDecodeError:
        continue

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  


### Challenge 15

In [21]:
def unpad(msg):
    if msg[-1] > 16 or msg[-msg[-1]:] != bytes([msg[-1]] * msg[-1]):
        raise Exception('Bad padding')
    return msg[:-msg[-1]]

In [22]:
print(unpad(b"ICE ICE BABY\x04\x04\x04\x04"))
try:
    unpad(b"ICE ICE BABY\x05\x05\x05\x05")
except Exception as e:
    print(e)
try:
    unpad(b"ICE ICE BABY\x01\x02\x03\x04")
except Exception as e:
    print(e)

b'ICE ICE BABY'
Bad padding
Bad padding


### Challenge 16

In [23]:
IV = random_bytes(16)

def enc1(str):
    str = str.replace(';', '%3b').replace('=', '%3d')
    str = "comment1=cooking%20MCs;userdata=" + str + ";comment2=%20like%20a%20pound%20of%20bacon"
    return cbc_encrypt(str.encode('ascii'), IV, cipher)

In [24]:
def dec1(ciph):
    return ";admin=true;" in cbc_decrypt(ciph, IV, cipher).decode('ascii', errors='replace')

In [32]:
ciphtxt = bytearray(enc1('00000xadminxtrue'))
ciphtxt[16:32] = xor(ciphtxt[16:32], xor(b'00000x00000x0000', b'00000;00000=0000'))
print(cbc_decrypt(bytes(ciphtxt), IV, cipher).decode('ascii', errors='replace'))
dec1(bytes(ciphtxt))

comment1=cookingD;CT����(K_�]00000;admin=true;comment2=%20like%20a%20pound%20


True